### Materials Project Workshop – August 8–10 2018, Berkeley, California\n",
#### Link to notebook: http://workshop.materialsproject.org/pymatgen/core/pymatgen_core.ipynb

# Pymatgen Adventures


This notebook will go over advanced pymatgen functionalities. We will build upon the core concepts in the introduction to pymatgen and use Mateirals Project data to perform materials analysis.



## 1. Building a phase diagram
Using the information we've learn about MPRester and Pymatgen, let's discuss an example of building a phase diagram. We are going to make the phase diagrams for Nickel, Oxygen and all of their combinations. 

Since, we will be plotting a phase diagram in a jupyter-notebook, we can run a command so that they show up a little cleaner:

In [ ]:
%matplotlib inline

Pymatgen has a built in interface to the Materials Project called the MPRester. This simplifies the process of getting data and transforming it into pymatgen objects so that we can jump right into performing analysis.

In [ ]:
from pymatgen import MPRester
mpr = MPRester()

Now, let's use the MPRester to get all phase diagram entries in the Ni-O chemical system

In [ ]:
system = ["Ni", "O"]
entries = mpr.get_entries_in_chemsys(system)

In [ ]:
print("Total Entries: ", len(entries),"\n\n")

print(entries[0])

Next, we use these entries to create a Phase Diagram and plot it. The plot below will show the formation energy vs the fraction of compound

In [ ]:
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.analysis.phase_diagram import PDPlotter


pd = PhaseDiagram(entries)
plotter = PDPlotter(pd)
plotter.show()

Notice that not all the entries were plotted. We have compounds at the terminal compositions (the elements), and two in between. This is a compositional phase diagram. Since we used Materials Project data, which is computed using Density Functional Theory (DFT) at 0K, we have produced a 0K Phase diagram.

What if we want to see the full phase diagram with all the unstable entries?

In [ ]:
plotter = PDPlotter(pd,show_unstable=True)
plotter.show()

Now, we see all the compositions including those that are unstable

### 1.1 Add our own Entry into the phase diagram

The phase diagram is a usefull tool to understand phase stability. Often we want to take MP data and see what the phase stability of our own calculated materials are. We start off by making an entry for our "computed" material

In [ ]:
from pymatgen.entries.computed_entries import ComputedEntry

my_entry = ComputedEntry(composition="Ni4O2",
                  energy=-28,
                  parameters={"potcar_symbols": ['pbe Ni_pv', 'pbe O'],
                              "hubbards":{'Ni': 6.2, 'O': 0.0}},
                  data={"oxide_type":"oxide"})

print(my_entry)

Now, we have to process it through the MateiralsProjectCompatability to make sure it gets the right corrections. MP has a number of corrections that ensure well measured reaction energies are properly recreated. This accounts for elemental biases such as the overbinding of electrons to oxygen in DFT.

In [ ]:
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
compat = MaterialsProjectCompatibility()
compat.process_entries([my_entry])

### 1.2 Exercise: Recreate the phase diagram with the new entry added in

Now that we've seen the way we can add our own entry to the phase diagram, let's try to recreate the plot with the new entry added in! 

```
new_entries = list(entries)

new_entries.extend(_______)
pd = ________ 
plotter = ________ 
_____________ 
```

In [ ]:
#Answer
new_entries = list(entries)

new_entries.extend(compat.process_entries([my_entry]))
pd = PhaseDiagram(new_entries)
plotter = PDPlotter(pd)
plotter.show()

## 2. Plotting Electronic Structure

The Materials Project contains over 50,000 pre-computed bandstructures and densities of states. This data can then be used to calculate seeebeck coefficeints understand defects and optical transitions. For this example, we'll perform the simple task of getting a bandstructure and plotting it

Go onto materials project and find a material that has a bandstructure.
Then we'll grab the bandstructure object for that material

In [ ]:
bs = mpr.get_bandstructure_by_material_id("mp-2657")
print(bs)


Notice that the pymatgen rest module gave us a BandStructureSymmLine object
This is a specific type of bandstructure object that traces an important path in the BZ to identify properties such as the band gap

Let's check the band gap

In [ ]:
print(bs.get_band_gap())

Now, let's plot the bandstructure

In [ ]:
from pymatgen.electronic_structure.plotter import BSPlotter

plotter=BSPlotter(bs)
plotter.get_plot().show()

Sometimes it's useful to visualize what this path is:

In [ ]:
plotter.plot_brillouin()


There is a similar method to get the density of states for a material.


In [ ]:
dos = mpr.get_dos_by_material_id("mp-2657")
print(dos)

Let's now get the elemental components. This is TiO2, so there is a DOS for Ti and a DOS for O

In [ ]:
dos_elemns = dos.get_element_dos()
print(dos_elemns)

Let's also get the orbitially seperated DOS for Ti

In [ ]:
dos_ti = dos.get_element_spd_dos("Ti")
print(dos_ti)

Let's plot the total DOS

In [ ]:
from pymatgen.electronic_structure.plotter import DosPlotter
dp = DosPlotter()
dp.add_dos("Total",dos)
dp.get_plot().show()

### 2.1 Exercise: Plot the total DOS for the TiO2 Elemental DOS'es shown above

Plot the total and elemental DOSes all together

```
dp = _______
for el,e_dos in dos_elemns.items():
    ________
    
___________
```

In [ ]:
# Answer
dp = DosPlotter()
for el,e_dos in dos_elemns.items():
    dp.add_dos(str(el),e_dos)
    
dp.get_plot().show()

## 3. Find the best substrate for BiFeO3

Another tool in pymatgen is called the SubstrateAnalyzer. This tool uses geometry and continuum elasticity to find the best substrate to grow a material on. We'll use this to find the best substrate for BiFeO3


First, let's the BiFeO3 structure

In [ ]:
film = mpr.get_structure_by_material_id("mp-645159")

print(film)

Next, we need to get the elastic tensor. Pymatgen has a built in Tensor defintion and ElasticTensor definition. This is usefull when you want to perform some sort of tensor operation such as rotating or trasnforming it, or even applying it to a structure. 

Let's use MPRester to grab the elasticity data for BiFeO3

In [ ]:
elac = mpr.query({"material_id": "mp-645159"},['elasticity'])[0]

elac

Now, let's convert it into an Elastic Tensor object

In [ ]:
from pymatgen.analysis.elasticity import ElasticTensor
tensor = ElasticTensor.from_voigt(elac['elasticity']['elastic_tensor'])
tensor.voigt

Let's get a list of substrates to consider. Materials Project has collected a set of the most accessible substrates and their corresponding MPids

In [ ]:
mpr.get_all_substrates()[:10]

We'll use some fancy python to get all the structures corresponding to these materials ids

In [ ]:
pot_substrates = [mpr.get_structure_by_material_id(s) for s in mpr.get_all_substrates()[:10]]
print(pot_substrates)

Next we need to initialize the substrate analyzer

In [ ]:
from pymatgen.analysis.substrate_analyzer import SubstrateAnalyzer
sa = SubstrateAnalyzer()

The analyzer can be used to calculate matches between a substrates and films. We need to go through those matches and add in any extra metadata and store those matches

In [ ]:
all_matches = []
for substrate in pot_substrates:
    matches = list(sa.calculate(film=film,substrate=substrate,elasticity_tensor=tensor,lowest=True))
    for m in matches:
        m['composition'] = str(substrate.composition.reduced_formula)
    all_matches.extend(matches)

In [ ]:
len(all_matches)

Let's plot the matches

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

for i in all_matches:
    plt.scatter(i['elastic_energy'],i['match_area'])
plt.xlabel("Elastic Energy (eV)")
plt.ylabel(r"Matching Area ($\AA^2$)")
plt.xlim(0,0.005)
plt.show()

Our previous analysis shows that the lower left corner is preferred. So let's look at matches with matching area < 50.0

In [ ]:

for m in all_matches:
    if m['match_area'] < 50.0 and m['elastic_energy'] < 0.002:
        print("Material: {} Substrate Face: {}  Film Direction: {}  Strain: {}".format(m['composition'],
                                                                                      m['sub_miller'],
                                                                                      m['film_miller'],
                                                                                      m['strain']))